# Linearity test using sky background - SITCOM-510

This notebook contains the execution for the linearity tests using sky background described in https://jira.lsstcorp.org/browse/SITCOM-510.
                
The notebook is organized in 6 sections:

    1.1 Setup
    1.2 Conditions assessment
    1.3. Choose target from list of targets in ticket. 
    1.4. CWFS nearby
    1.5. Slew to target object.
    1.6  Perform the data acquisition sequence. 
    
    
Warning! This test must start at the beginning of the astronomical dawn (Sun at 18 deg below the horizon), to allow enough time to perform CWFS and slew to the target, and commence the data sequence at -15 degree morning twilight. 

End of dark time- Beginning of astronomical twilight (Sun at -18 deg) for November's observing nights:
Tue 8th - 05:18
Wed 9th - 05:17
Thu 10th - 05:16

Tue 22nd - 05:06
Wed 23rd - 05:05
Thu 24th - 05:05

----
## Assess that conditions meet criteria
This must be performed during morning twilight, and requires photometric conditions.

Best to do this with fixed shutter-open time so we don’t introduce any shutter artifacts. Also best to do this during morning twilight so that any dome seeing has settled down and seeing is stable.

If we begin this entire operation at around 15 degree morning twilight then we should be able to run until sun is something like (this is a guess) 8-10 degrees below horizon, so we should get 20-30 minutes of data from this. That should be enough for 10 cycles through the data acquisition loop.  

-----
## Setup

### Import libraries

In [1]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

### Setting up logger

In [2]:
logger = logging.getLogger("SITCOM-510")
logger.level = logging.DEBUG

### Getting unique index for script

In [3]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

### Instantiate CWFS Script

In [4]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [5]:
atcs = script.atcs
latiss = script.latiss

### Set up script log level

In [6]:
script.log.level = logging.DEBUG

### Write start info into EFD

In [7]:
script.log.info(f'START- SITCOM-510 Linearity test using sky background -- at {Time.now()}')

----
## Target and filter

### Declare target: Choose target - Fields need to be reasonably dense with stars. 
Fields appropriate for this, for morning twilight for each month of the year, are listed in Table 1 of the PDF attached in https://jira.lsstcorp.org/browse/SITCOM-510

In [8]:
targets = {'Nov': {'target_name':'LinTest0513' , 'target_ra' : '05:13:36.52', 'target_dec' :'-40:02:44.2'}, 
          'Dec': {'target_name' :'LinTest0800', 'target_ra': '08:10:00' , 'target_dec' : '-36:10:00'},
          'Jan': {'target_name' :'LinTest1000', 'target_ra': '10:00:00' , 'target_dec' : '-45:20:00'},
          'Feb': {'target_name' :'LinTest1200', 'target_ra': '12:00:00' , 'target_dec' : '-50:30:00'},
          'Mar': {'target_name' :'LinTest1400', 'target_ra': '14:00:00' , 'target_dec' : '-55:00:00'},
          'Apr': {'target_name' :'LinTest1600', 'target_ra': '16:00:00' , 'target_dec' : '-28:30:00'},
          'May': {'target_name' :'LinTest1800', 'target_ra': '18:00:00' , 'target_dec' : '-25:00:00'},
          'Jun': {'target_name' :'LinTest2000', 'target_ra': '20:00:00' , 'target_dec' : '-36:10:00'},
          'Jul': {'target_name' :'LinTest2133', 'target_ra': '21:33:02.06' , 'target_dec' : '-00:50:11.7'},
          'Aug': {'target_name' :'LinTest2308', 'target_ra': '23:08:10.65' , 'target_dec' : '-15:37:13.7'},
          'Sep': {'target_name' :'LinTest0052', 'target_ra': '00:52:45.55' , 'target_dec' : '-26:28:00.4'},
          'Oct': {'target_name' :'LinTest0311', 'target_ra': '03:11:42.23' , 'target_dec' : '-55:18:08.8'},
          }

Declare the month of observation and confirm that the target is correct. 

In [9]:
month = 'Nov'

In [10]:
logger.info(f'The target name is {targets[month]["target_name"]}, \n'
            f'target RA is {targets[month]["target_ra"]} & target DEC is {targets[month]["target_dec"]},\n'
            f'Month of observation is {month}')

### Declare filter - Set up with reddest band in filter wheel (to get best image quality) and no disperser

In [35]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'Available filters are: {inst_setup[0]},\n Gratings are: {inst_setup[1]}')

In [11]:
filter_to_use = 'SDSSr_65mm'

------
## CWFS
A CWFS is recommended just before the test to ensure the system is in focus. A bright source near the target will be selected to perform CWFS on. 

### Declare CWFS target 

Query for a source around the same area of the sky as the target source.

In [22]:
target_azel= atcs.azel_from_radec(ra=targets[month]["target_ra"], dec=targets[month]["target_dec"])

In [24]:
cwfs_az = target_azel.az.degree
cwfs_el = target_azel.alt.degree
cwfs_mag_limit = 8
cwfs_radius= 4

In [25]:
cwfs_target = await script.atcs.find_target(az=cwfs_az, el=cwfs_el, mag_limit=cwfs_mag_limit, radius = cwfs_radius)
logger.info(f'Target for CWFS with magnitude limit {cwfs_mag_limit} is {cwfs_target}')

### Slew to the CWFS target

In [26]:
await script.atcs.slew_object(cwfs_target, rot_type=RotType.PhysicalSky)

Uncomment the following line to take a snapshot to verify the target is positioned as expected 

In [ ]:
# exposure = await latiss.take_acq(
#         exptime=5, n=1, filter=filter_to_use, grating='empty_1', reason='Acquisition', program="SITCOM-510")
# logger.info(f'Acquisition exposure is {exposure}')

To offset the telescope and center the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

### Set up configuration

In [27]:
configuration = yaml.safe_dump({"filter": filter_to_use, 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "program" : "SITCOM-510"})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [28]:
await script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [29]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [30]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script
This sets the same group ID for all exposures taken in the script.

In [31]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1070028783, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Hexapod correction is not enabled. Offsets cannot be applied. See the enableCorrection to close the correction loop.')

-------
## Slew to target object

### Confirm that the target is correct

In [32]:
logger.info(f'Target is {targets[month]["target_name"]}')

### Slew to the target object

In [36]:
await script.atcs.slew_icrs(ra = targets[month]['target_ra'], dec = targets[month]['target_dec'], rot_type = RotType.PhysicalSky)

(<ICRS Coordinate: (ra, dec) in deg
     (78.40216667, -40.04561111)>,
 <Angle 0. deg>)

To offset the telescope and move the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
#await script.atcs.offset_xy(x=20, y=20)

----
## Data Acquisition Sequence: Once sky brightness level reaches 1000 ADU above bias level in a red band (SDSSr) in a 10 second exposure, begin the following sequence, to be looped until sky reaches saturation in a 5 second image:  

###  Check signal level: 
Take a 10-s image to check that the sky brightness is around 1000 ADU above bias level, that is around 13000 ADUs. 
To check signal levels, use the new CCS feature http://ccs.lsst.org/RecentImages/auxtel.html. As you hover over the Simple Overscan Correction` image, readings in true counts will displayed on the bottom right. 

In [ ]:
sky_brightness_test = await latiss.take_engtest(
        exptime=10, n=1, filter=filter_to_use, grating='empty_1', reason='Sky_Brightness_Level_Test', program = "SITCOM-510")
logger.info(f'Sky Brightness test exposure is {sky_brightness_test}')

### Data Acquisition Loop
This will run for 10 iterations. In case saturation is detected in the 5sec image, you may stop the execution manually. Otherwise, repeat the loop until saturation is seen. 

In [ ]:
i=1
while i<11:
    logger.info(f'Loop cycle number {i}')
    
    darks_5sec = await latiss.take_darks(
        exptime=5, ndarks=3, reason='5sec_Darks_for_Linearity_using_sky_background', program ="SITCOM-510")
    logger.info(f'5sec darks in loop {i} are {darks_5sec}')
    
    exp_5sec = await latiss.take_engtest(
        exptime=5, n=3, filter=filter_to_use, grating='empty_1', reason='5sec_Images_for_Linearity_using_sky_background', program ="SITCOM-510")
    logger.info(f'5sec images in loop {i} are {exp_5sec}')
    
    exp_10sec = await latiss.take_engtest(
        exptime=5, n=3, filter=filter_to_use, grating='empty_1', reason='10sec_Images_for_Linearity_using_sky_background', program ="SITCOM-510")
    logger.info(f'10sec images in loop {i} are {exp_10sec}')
    
    exp_30sec = await latiss.take_engtest(
        exptime=30, n=3, filter=filter_to_use, grating='empty_1', reason='30sec_Images_for_Linearity_using_sky_background', program ="SITCOM-510")
    logger.info(f'30sec images in loop {i} are {exp_30sec}')
    
    i +=1 

### Write end info into EFD

In [ ]:
script.log.info(f'END- SITCOM-510 Linearity test using sky background -- at {Time.now()}')